## **Chatbot Running Llama and LangChain**

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.1 

### **Loading Llama7B**

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch
import re

In [3]:
MODEL_NAME = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

### **Run Llama model**

In [5]:
def run_model(question):
  device = "cuda:0"
  inputs = tokenizer(question, return_tensors="pt").to(device)
  response = model.generate(**inputs, max_new_tokens=10)
  result = tokenizer.decode(response[0], skip_special_tokens=True)

  return re.sub("\\n", " ", result)

In [6]:
question = "What is the capital of England?"
run_model(question)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1296: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


'What is the capital of England?  nobody knows. What is the capital of'

### **Run Llama with Langchain**

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [22]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

In [23]:
question = "What is the capital of England?"

print(llm_chain.run(question))

 London, United Kingdom



In [24]:
question = "Write a Python function to calculate the factorial of a number."

print(llm_chain.run(question))

 \begin{code}
def fact(n):
    if n == 0 or n==1 : return 1;
    else :return (fact(n-1)*n)
\end{code}


In [20]:
from pprint import pprint

In [21]:
question = "Explain about diabetes"

pprint(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1089: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


(' Diabetes mellitus, commonly referred to as just "diabetes", is a group of '
 'metabolic diseases in which there are high blood sugar levels over a '
 'prolonged period. Symptoms often include frequent urination, increased '
 'thirst and increased appetite. If left untreated, long-term complications '
 'can include cardiovascular disease, stroke, chronic kidney failure, foot '
 'ulcers, and damage to eyes.\n'
 '\n'
 'Diabetes occurs either when pancreas does not produce enough insulin or when '
 'body cannot effectively use the insulin produced. This leads to high blood '
 'glucose level. There are two main types of diabetes - type 1 (insufficient '
 'production) and type 2 (effective utilization). Type I accounts for only 5% '
 'cases while rest 90% fall under category II.\n'
 '\n'
 'Type I diabetics require daily administration of insulin through injection '
 'or pump whereas those with type II may be able to control their condition by '
 'proper')


In [25]:
question = "who is Ronaldo?"

pprint(llm_chain.run(question))

('\n'
 'Ronaldo is a Brazilian professional footballer who plays as a forward for '
 'Italian club Juventus and captains the Portugal national team. Often '
 'considered the best player in the world and widely regarded as one of the '
 "greatest players of all time, he has won five Ballons d'Or (the first player "
 'to win three consecutive ones), four UEFA Champions Leagues, seven league '
 'titles, five FIFA Club World Cups, one European Championship and two Copa '
 'América trophies among several other honours. He holds many records; '
 'including most goals scored in Serie A (120) and the all-time top scorer for '
 'his club level appearances with 438 goals for Real Madrid, becoming the only '
 'player ever to achieve such milestone while playing exclusively for one '
 'single club. His total goal tally for club competitions stands at over 750 '
 'goals across nearly 900 official games during his career.\n')


In [26]:
question = "what do you do in your free time?"

pprint(llm_chain.run(question))

(' I like to play video games and watch movies with my friends on weekends, '
 'but during school days i usually spend most of my time studying for exams or '
 'doing homework assignments.')


In [27]:
question = "good do you like painting too?"

print(llm_chain.run(question))

 I love to paint! It's my favorite hobby and it helps me relax after work or school. 😊


In [28]:
question = "what brand of phone do you prefer have?"

pprint(llm_chain.run(question))

(' I like Samsung phones because they are very reliable and easy to use, but '
 "if it's not available in my country then I would go for Apple or Huawei. \n")
